## Visual Analytics 
### Data cleaning for the final Project 
##### Supervisors:  Sippo Rossi & Ivanna Jurkiv 

In [ ]:
#Importing important librarys
import pandas as pd 
import numpy as np
from tqdm import tqdm
from googletrans import Translator, constants

#Adding the progress bar capability to pandas using tqdm
tqdm.pandas()

In [8]:
# Read the Excel file
dfs = pd.read_excel('v0.1__data_climbing.xlsx', sheet_name=None)

In [9]:
# Access the sheets as a DataFrame
df_sub_region = dfs['Teilgebiet']
df_summit = dfs['Gipfel']
df_routes = dfs['Wege']
df_comments = dfs['Kommentare']
df_scales = dfs["scales"]

In [10]:
#renaming columns in dataframe
df_sub_region = df_sub_region.rename(columns = {"Sterne ":"stars",
                                                "SEKTORID":"sector_id",
                                                "SEKTORNAME_D":"sectorname"})
df_sub_region = df_sub_region.iloc[:, :-2] #remove last 2 columns
print(df_sub_region)

    sector_id             sectorname  stars
0         134                 Wehlen      3
1         130                 Rathen      5
2         128                  Brand      0
3         132          Schrammsteine      5
4         131               Schmilka      5
5         123            Affensteine      0
6         129        Kleiner Zschand      0
7         126         Großer Zschand      0
8         135   Wildensteiner Gebiet      0
9         127        Hinterhermsdorf      3
10        133      Gebiet der Steine      5
11        124               Bielatal      4
12        125  Erzgebirgsgrenzgebiet      0


In [11]:
#renaming columns
df_summit = df_summit.rename(columns = {"SEKTORID":"sector_id",
                                        "GIPFELID":"summit_id",
                                        "GIPFELNAME_D":"summit_name",
                                        "LAT":"gps_lat",
                                        "LONG":"gps_long",
                                        "Anzahl Wege": "count_routes",
                                        "GIPFELWERTUNG":"summit_type"})


df_summit = df_summit.iloc[:, :-3].dropna(how = "all") #remove last 3 columns and row if all "NaN"

#fixing messed up gps numbers (change to str for the next step)
df_summit["gps_lat"] = df_summit["gps_lat"].astype(str)
df_summit["gps_long"] = df_summit["gps_long"].astype(str)

#implementing a dot at the right position
df_summit["gps_lat"] = df_summit["gps_lat"].str.replace('.', '',regex=False).str[:2] + "," + df_summit["gps_lat"].str[2:-2].fillna(0)
df_summit["gps_long"] = df_summit["gps_long"].str.replace('.', '',regex=False).str[:2] + "," + df_summit["gps_long"].str[2:-2].fillna(0)

#gettig all summits for each sector 
df_summit["summit_counts"] = df_summit.groupby("sector_id")["sector_id"].transform("count")
df_summit["sector_id"] = df_summit["sector_id"].fillna(0)
df_summit["sector_id"] = df_summit["sector_id"].astype(int)

df_summit.loc[df_summit["summit_type"] > 5, "summit_type"] = np.nan  #cleaning the column

#merging on "sector_id" and dropping duplicates
df_sub_region = df_sub_region.merge(df_summit[["sector_id","summit_counts"]], on = "sector_id", how = "left").drop_duplicates()

print(df_summit.head())

   sector_id  summit_id  summit_name  summit_type        gps_lat  \
0        123     3437.0     Vagabund          NaN     50,9093088   
1        123     3438.0     Hallodri          NaN     50,9092927   
2        123     3439.0     Lehnhorn          2.0     50,9089816   
3        123     3440.0  Lehnwächter          3.0     50,9083915   
4        123     3441.0  Lorenzsporn          4.0  50,9094722222   

        gps_long  POSFEHLER  count_routes  summit_counts  
0     14,2197579       10.0           NaN            119  
1     14,2199028       10.0           NaN            119  
2     14,2203265       10.0           NaN            119  
3  14,2227222222       40.0           NaN            119  
4  14,2236111111       40.0           NaN            119  


In [12]:
print(df_summit.dtypes)

sector_id          int32
summit_id        float64
summit_name       object
summit_type      float64
gps_lat           object
gps_long          object
POSFEHLER        float64
count_routes     float64
summit_counts      int64
dtype: object


In [13]:
print(df_summit.groupby("sector_id").count())

           summit_id  summit_name  summit_type  gps_lat  gps_long  POSFEHLER  \
sector_id                                                                      
123              119          119           79      119       119        119   
124              249          249          122      249       249        249   
125               16           16           14       16        16         16   
126               86           86           60       86        86         86   
127               17           17           17       17        17         17   
128               84           84           51       84        84         84   
129               44           44           32       44        44         44   
130              148          148          103      148       148        148   
131              127          127           68      127       127        127   
132               81           81           63       81        81         81   
133              112          112       

In [14]:
#renaming columns
df_routes = df_routes.rename(columns = {"GIPFELID":"summit_id",
                                        "WEGID":"route_id",
                                        "WEGNAME_D":"route_name",
                                        "ERSTBEGEHERVORSTIEG":"first_ascent",
                                        "ERSTBEGEHERNACHSTIEG":"fa_group",
                                        "ERSTBEGEHUNGSDATUM":"fa_date",
                                        "WEGBESCHREIBUNG_D":"route_description_de",
                                        "STICHWORTE":"tags",
                                        "SCHWIERIGKEIT":"grade",
                                        "QUALITAET":"quality",
                                        "SICHERUNG":"safety",
                                        "NAESSE":"wet",
                                        "RINGZAHL":"safety_rings",
                                        "WEGSTATUS":"status_route"
                                       })
df_routes = df_routes.iloc[:, :-1] #remove last column

#remove routes where we don't have the summit for
df_routes = df_routes[df_routes['summit_id'].isin(df_summit['summit_id'])]

#count the routes_per_summit
df_routes["routes_per_summit"] = df_routes.groupby("summit_id")["summit_id"].transform("count")

print(df_routes.head())

   summit_id  route_id            route_name                    first_ascent  \
0       3437      1382             Alter Weg                  Wolfgang Poock   
1       3437     18682       Variante zum AW                    Günter Schär   
2       3437     92855               Westweg                   Manfred Vogel   
3       3437     65288           Pennermatte                   Volkmar Werth   
4       3437     18683  Südwestverschneidung  Jürgen Weber und Günter Müller   

      fa_group              fa_date  \
0         vug.  1959-11-01 00:00:00   
1  S. Hoffmann  1960-07-24 00:00:00   
2          vug  2017-05-10 00:00:00   
3    F. Bonitz  1994-07-07 00:00:00   
4          NaN  1960-08-07 00:00:00   

                                route_description_de tags grade  quality  \
0  Vom Massiv absteigen und durch Kamin zur O-Kan...  NaN    IV      0.0   
1  Links in der N-Wand Kamin in Winkel einige m h...  NaN    IV      0.0   
2                                                NaN  Na

In [15]:
#get only routes which we have a summit for
df_routes = df_routes[df_routes['summit_id'].isin(df_summit['summit_id'])]

#star at the beginning of the name indicates beautiful route
df_routes["beautiful"] = df_routes["route_name"].str.count("^\*")
df_routes["route_name"] = df_routes["route_name"].str.replace("^\*", "",regex=True)

#transforming fa_date elements
df_routes["fa_date"] = df_routes["fa_date"].astype(str)
df_routes["fa_date"] = df_routes["fa_date"].replace("0000-00-00", "0")
df_routes["fa_date"] = df_routes["fa_date"].apply(lambda x: x[:-9] if x.endswith(" 00:00:00") else x)
print(df_routes.head())

   summit_id  route_id            route_name                    first_ascent  \
0       3437      1382             Alter Weg                  Wolfgang Poock   
1       3437     18682       Variante zum AW                    Günter Schär   
2       3437     92855               Westweg                   Manfred Vogel   
3       3437     65288           Pennermatte                   Volkmar Werth   
4       3437     18683  Südwestverschneidung  Jürgen Weber und Günter Müller   

      fa_group     fa_date                               route_description_de  \
0         vug.  1959-11-01  Vom Massiv absteigen und durch Kamin zur O-Kan...   
1  S. Hoffmann  1960-07-24  Links in der N-Wand Kamin in Winkel einige m h...   
2          vug  2017-05-10                                                NaN   
3    F. Bonitz  1994-07-07  4 m rechts der "Südwestverschneidung" teils üb...   
4          NaN  1960-08-07  In Mitte SW-Wand zu Riss und Verschneidung zum...   

  tags grade  quality  safety  w

In [18]:
#getting routes_counts summed by summit_id 

#creating new dataframes 's_df' and 'b_df'
s_df = df_routes[["summit_id","route_id","routes_per_summit"]]
b_df = df_summit[["summit_id","sector_id","summit_counts"]]
b_df["summit_id"] = b_df["summit_id"].astype(int)

# Merging on 'summit_id' column to create 'm_db'
m_db = b_df.merge(s_df, left_on = "summit_id", right_on = "summit_id")

#calculating 'routes_counts_per_sec' for each sector
m_db["routes_counts_per_sec"] = m_db.groupby("sector_id")["sector_id"].transform("count")

#merging on 'sector_id' column, and dropping duplicates
df_sub_region = df_sub_region.merge(m_db[["sector_id","routes_counts_per_sec"]], on = "sector_id", how = "left").drop_duplicates()

In [19]:
print(df_sub_region)

       sector_id             sectorname  stars  summit_counts  \
0            134                 Wehlen      3             18   
289          130                 Rathen      5            148   
2986         128                  Brand      0             84   
4819         132          Schrammsteine      5             81   
6509         131               Schmilka      5            127   
8900         123            Affensteine      0            119   
11060        129        Kleiner Zschand      0             44   
12223        126         Großer Zschand      0             86   
13873        135   Wildensteiner Gebiet      0             42   
15324        127        Hinterhermsdorf      3             17   
15527        133      Gebiet der Steine      5            112   
18367        124               Bielatal      4            249   
22634        125  Erzgebirgsgrenzgebiet      0             16   

       routes_counts_per_sec_x  routes_counts_per_sec_y  routes_counts_per_sec  
0       

In [20]:
#setting display rows, to see how the grades are formated
pd.set_option('display.max_rows', 5000000)
#fix the grades
sgrade_list = df_scales["SÄCHSISCH"].tolist()
df_routes['grade'] = df_routes['grade'].astype(str).apply(lambda x: x.split(" ")[0] if x.split(" ")[0] in sgrade_list else x)

#simplifying/fixing the formating of the grade
for i, row in df_routes.iterrows():
    if i not in sgrade_list:
        if '/' in row['grade']:
            #split string and check if second part is in short_list
            split_str = row['grade'].split('/')
            if split_str[1] in sgrade_list or split_str[1].split(" ")[0] in sgrade_list:
                #replace element with short version
                df_routes.at[i, 'grade'] = split_str[1]
            

In [21]:
#function to replace the values in a column
def replace_all(col, dic):
    for i, j in dic.items():
        col = col.replace(i, j)
    return col

In [22]:
#replace values in 'grade' column
dic = {"1":"I","3":"III","4":"IV","2":"II","5":"V","6":"IV","7":"IIV"}
df_routes["grade"] = replace_all(df_routes["grade"],dic)

#keep values in 'grade' column in 'sgrade_list'
df_routes['grade'] = df_routes['grade'].apply(lambda x: x if x in sgrade_list else "NaN")

In [23]:
#checking the transfomation 
print(df_routes["grade"].value_counts().sort_index(ascending = False))

Xc         89
Xb        189
Xa        369
XIc        10
XIb        17
XIa        32
XIIc        9
XIIb       32
XIIa       48
VIIc     2182
VIIb     2197
VIIa     2193
VIIIc    1122
VIIIb    1368
VIIIa    1527
VI       1911
V        2164
NaN       187
IXc       518
IXb       724
IXa       839
IV       2027
III      1739
II       1036
I         369
Name: grade, dtype: int64


In [28]:
print(df_scales.head())

  UIAA FRZ. SÄCHSISCH BRITISH (TECH) BRITISH (ADJ) USA (SIERRA) NORDIC (FIN)  \
0    1    1         I              1             M            5            1   
1    2    2        II              2           M/D          5.1            2   
2    3    3       III              3             D          5.2          2/3   
3    4    4        IV             4a          D/VD          5.3            3   
4   4+   4+      IV/V             4a            VD          5.4            4   

  NORDIC (SWE/NOR) FB-SKALA (BOULDERN) V-SCALE (BOULDERN)  
0                1                   1                VB-  
1                2                   1                VB-  
2              2/3                 1/2                VB-  
3                3                   2                VB-  
4                4                   2                VB-  


In [29]:
#renaming columns
df_comments = df_comments.rename(columns = {"GIPFELID":"summit_id",
                                        "WEGID":"route_id",
                                        "SEKTORID":"sector_id",
                                        "KOMMENTID":"comment_id",
                                        "SCHWIERIGKEIT":"grade_maybelater",
                                        "QUALITAET":"quality",
                                        "SICHERUNG":"safety",
                                        "NAESSE":"wet",
                                        "DATUM":"date_comment",
                                        "KOMMENTAR":"comment_de"
                                       })
print(df_comments.head())

   sector_id  summit_id  route_id  comment_id  \
0        123       3437      1381       28277   
1        123       3438      1383       28278   
2        123       3439      1389       28498   
3        123       3439      1390       28281   
4        123       3441      1399       39174   

                                          comment_de        date_comment  \
0  Ein echtes Kleinod, das mensch sich allerdings... 2017-06-12 21:31:00   
1  Der Einstieg bis zum Ring ist extrem unangeneh... 2017-06-12 21:34:53   
2  Immer noch ein hübscher Weg, allerdings sollte... 2017-07-03 14:27:56   
3  Eine tolle Linie mit Anspruch. Zum ersten Ring... 2017-06-12 21:41:58   
4  Den Aufstieg auf den Pfeiler lieber von links ... 2021-05-25 22:23:23   

   quality  safety  grade_maybelater  wet  
0      1.0     2.0               6.0  NaN  
1      2.0     3.0               9.0  4.0  
2      2.0     3.0              10.0  NaN  
3      2.0     3.0              11.0  NaN  
4      3.0     3.0          

In [30]:
#initialize translator object
translator = Translator(service_urls=['translate.googleapis.com'])

#function to translate comments into english
def translate_comments(df_comments):
    en_com = []
    for comment in tqdm(df_comments["comment_de"].to_list()): #iterate through german comments
        en_com.append(translator.translate(comment).text) #translate and append to list "en_com"
    return en_com

#translating the comments and create new column
df_comments["comment_en"] = translate_comments(df_comments)

  2%|█▏                                                                              | 69/4476 [00:12<13:06,  5.60it/s]


KeyboardInterrupt: 

In [31]:
#removing some columns
del df_routes["routes_per_summit"]
df_summit = df_summit.drop(columns = ["count_routes", "summit_counts"])

#export 
writer = pd.ExcelWriter("climbing_data_25_01.xlsx", engine ="xlsxwriter")
df_scales.to_excel(writer, sheet_name = "scales", index=False)
df_routes.to_excel(writer, sheet_name = "routes", index=False)
df_summit.to_excel(writer, sheet_name = "summits", index=False)
df_sub_region.to_excel(writer, sheet_name = "sub_regions", index=False)
df_comments.to_excel(writer, sheet_name = "comments", index=False)
writer.save()